Data Loading


In [32]:
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error


In [3]:
apple_df = pd.read_csv('AAPL.csv')

In [ ]:
apple_df.shape

In [ ]:
apple_df.head()

Data Pre-Processing

In [ ]:
#checking for null values
apple_df.isna().sum()

In [ ]:
apple_df.info()

In [8]:
#converting the dataype of 'Date' col to 'datetime'
apple_df['Date'] = pd.to_datetime(apple_df['Date'])

In [9]:
#making the 'Date' col as index
apple_df.set_index('Date', inplace = True)

In [ ]:
apple_df.info()

In [11]:
#sort the indexes
apple_df.sort_index(inplace = True)

In [ ]:
apple_df.head()

In [13]:
# Apply log transformation to Volume to stabilize variance
apple_df['Log_Volume'] = np.log(apple_df['Volume'])

In [14]:
#deleting the original column of volume
apple_df.drop(columns = ['Volume'], inplace = True)

In [ ]:
apple_df.head()


In [16]:
#normalizing the data
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(apple_df[apple_df.columns])

In [ ]:
scaled_values

In [18]:
#converting the array into dataframe
apple_scaled_df = pd.DataFrame(scaled_values, columns = apple_df.columns, index = apple_df.index)

In [ ]:
apple_scaled_df.head()

Plotting the Column


In [ ]:
plt.rcParams['figure.figsize'] = (15, 15)
figure, axes = plt.subplots(6)

for ax, col in zip(axes, apple_scaled_df.columns):
  ax.plot(apple_scaled_df[col])
  ax.set_title(col)
  ax.axes.xaxis.set_visible(False)

Creating the sliding window sequence

In [21]:
window_size = 60

def create_sequence(data, window_size):
    X = []
    y = []
    for i in range(window_size, len(data)):
        X.append(data.iloc[i-window_size:i].values)
        y.append(data.iloc[i].values)
    return np.array(X), np.array(y)

X, y = create_sequence(apple_scaled_df, window_size)

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train.shape[1], X_train.shape[2], y_train.shape[1]

Building the LSTM Model


In [ ]:
model = keras.Sequential([
    # Adding the first LSTM layer with Dropout
    keras.layers.LSTM(units = 50, return_sequences=True, input_shape = (X_train.shape[1], X_train.shape[2])),
    keras.layers.Dropout(0.3),

    # Adding the second LSTM layer with Dropout
    keras.layers.LSTM(units = 50, return_sequences=True),
    keras.layers.Dropout(0.3),

    # Adding the third LSTM layer with Dropout
    keras.layers.LSTM(units = 50, return_sequences=False),
    keras.layers.Dropout(0.3),

    # Adding a Dense output layer
    keras.layers.Dense(y_train.shape[1])
])

In [26]:
#compilation
model.compile(optimizer = 'adam',
              loss = 'mean_squared_error',
              metrics = ['RootMeanSquaredError'])

In [27]:
#Early stopping condition
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100,
                    batch_size=3,
                    callbacks=[early_stopping])

In [ ]:
# Making predictions on the test data
predictions = model.predict(X_test)

In [30]:
# Inverse scaling to get the original values
predictions = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test)

In [ ]:
# Plotting the close column predictions
plt.figure(figsize=(10, 5))

# Assuming the 'close' column is at index 0 in your dataset
col_index = apple_scaled_df.columns.get_loc("close")  # Get the index of 'close' column

# Plot the actual and predicted values for the 'close' column
plt.plot(y_test_rescaled[:, col_index], color='blue', label='Actual Close Price')
plt.plot(predictions[:, col_index], color='red', label='Predicted Close Price')

# Add titles and labels
plt.title('Close Price Prediction')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.legend()

plt.show()


In [ ]:

# RMSE Calculation



# Assuming the 'close' column is at index 0 in your dataset
col_index = apple_scaled_df.columns.get_loc("close")  # Index of the 'close' column

# Extract actual and predicted values for the 'close' column
actual_close = y_test_rescaled[:, col_index]
predicted_close = predictions[:, col_index]

# Calculate RMSE
rmse_close = np.sqrt(mean_squared_error(actual_close, predicted_close))

# Calculate Mean of Actual Close Prices
mean_actual_close = np.mean(actual_close)

# Accuracy in percentage
accuracy_close = (1 - (rmse_close / mean_actual_close)) * 100

# Display results
print(f"Close Price RMSE: {rmse_close:.2f}")
print(f"Close Price Prediction Accuracy: {accuracy_close:.2f}%")
